# P1. Exploration

## 1. Description Générale

En utilisant DBeaver et les paramètres de connexion donnés par ailleurs, faire une description détaillée de la base de données fournie. Présenter notamment le diagramme entité - relation (ERD) et les différentes caractéristiques des tables de la base.  Quelles types de renseignement peut-on tirer de cette base de données ?   

*A compléter*

---

## 2. Exploration SQL via python / pandas
Dans cette partie, on s'attache à explorer la base de données en utilisant le language SQL pour les requêtes et le language Python (+ module pandas) pour récupérer les résultats de façon programmatique.  
Vous pouvez utiliser DBeaver ou outil similaire pour préparer / tester vos requêtes mais celles-ci doivent être reportées et appliquées dans du code Python comme présenté dans les quelques examples à suivre.  
*Pour les résultats "longs", un aperçu des quelques premières lignes et le comptage total des résultats est suffisant.*

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

### Organigramme Compagnie

Q01. Ecrivez et exécutez une unique requête listant chaque employée de la compagnie (Id, nom + prénom, Titre) ainsi que son manager direct (nom + prénom, Titre)

In [12]:
# requête de départ
query = """
    SELECT e.EmployeeId AS 'Employee Id', 
           e.firstname || ' ' || e.lastname AS 'Employee',
           e.Title AS 'Titre',
           e.ReportsTo AS 'Reports to Id',
           -- self join on considère qu'on a une table manager qui dérive de la table employee
       m.EmployeeId AS 'Manager Id', 
           m.firstname || ' ' || m.lastname AS 'Manager',
           m.Title AS 'Titre'    
FROM employees e
LEFT JOIN employees m ON e.ReportsTo = m.EmployeeId 
"""

In [13]:
results = pd.read_sql_query(query, conn)
results

,Employee Id,Employee,Titre,Reports to Id,Manager Id,Manager,Titre
0,1,Andrew Adams,General Manager,NaN,NaN,None,None
1,2,Nancy Edwards,Sales Manager,1.0,1.0,Andrew Adams,General Manager
2,3,Jane Peacock,Sales Support Agent,2.0,2.0,Nancy Edwards,Sales Manager
3,4,Margaret Park,Sales Support Agent,2.0,2.0,Nancy Edwards,Sales Manager
4,5,Steve Johnson,Sales Support Agent,2.0,2.0,Nancy Edwards,Sales Manager
5,6,Michael Mitchell,IT Manager,1.0,1.0,Andrew Adams,General Manager
6,7,Robert King,IT Staff,6.0,6.0,Michael Mitchell,IT Manager
7,8,Laura Callahan,IT Staff,6.0,6.0,Michael Mitchell,IT Manager


Q02. Pour chaque artiste présent dans cette BDD, indiquez le nombre d'albums de cet artiste également répertorié (Liste triée par nom d'artiste)


In [16]:
query = """SELECT DISTINCT(a.ArtistId) AS 'ID', a.Name, COUNT(a2.Title) as "Nombre d'album"
FROM artists a
JOIN albums a2 ON a.ArtistId = a2.ArtistId 
GROUP BY a.Name
LIMIT 10"""

In [17]:
results = pd.read_sql_query(query, conn)
results

,ID,Name,Nombre d'album
0,1,AC/DC,2
1,230,Aaron Copland & London Symphony Orchestra,1
2,202,Aaron Goldberg,1
3,214,Academy of St. Martin in the Fields & Sir Nevi...,1
4,215,Academy of St. Martin in the Fields Chamber En...,1
5,222,"Academy of St. Martin in the Fields, John Birc...",1
6,257,"Academy of St. Martin in the Fields, Sir Nevil...",1
7,2,Accept,2
8,260,Adrian Leaper & Doreen de Feis,1
9,3,Aerosmith,1


Q03. Produire une liste des albums reprenant également l'artiste de l'album, le nombre de pistes, la durée de toutes les pistes (en minutes) et le coût total (Liste triée par artiste puis nom d'album)


In [28]:
query = """SELECT a.Title AS 'Titre Album',  
		a2.Name,
		COUNT(t.TrackId) AS 'Nombre de piste', SUM(t.Milliseconds)/1000/60 AS 'Durée',
		SUM(t.UnitPrice) AS 'Prix' 
FROM albums a
LEFT JOIN artists a2 ON a.ArtistId = a2.ArtistId 
JOIN tracks t ON a.AlbumId = t.AlbumId 
GROUP BY a.AlbumId  
LIMIT 10"""

In [29]:
results = pd.read_sql_query(query, conn)
results

,Titre Album,Name,Nombre de piste,Durée,Prix
0,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,Balls to the Wall,Accept,1,5,0.99
2,Restless and Wild,Accept,3,14,2.97
3,Let There Be Rock,AC/DC,8,40,7.92
4,Big Ones,Aerosmith,15,73,14.85
5,Jagged Little Pill,Alanis Morissette,13,57,12.87
6,Facelift,Alice In Chains,12,54,11.88
7,Warner 25 Anos,Antônio Carlos Jobim,14,48,13.86
8,Plays Metallica By Four Cellos,Apocalyptica,8,44,7.92
9,Audioslave,Audioslave,14,65,13.86


Q04. Produire une liste des toutes les playlists reprenant son nom, le nombre de pistes, la durée de toutes les pistes (en minutes) et le coût total (Liste triée par nom de playlist)


In [31]:
query = """SELECT p.PlaylistId, p.Name, COUNT(pt.TrackId) AS 'Nombre piste',
SUM(t.Milliseconds/60000) AS 'Durée', SUM(t.UnitPrice) AS 'Prix Total'
FROM playlists p 
INNER JOIN playlist_track pt ON p.PlaylistId = pt.PlaylistId 
INNER JOIN tracks t ON pt.TrackId = t.TrackId 
GROUP BY p.PlaylistId """

In [32]:
results = pd.read_sql_query(query, conn)
results

,PlaylistId,Name,Nombre piste,Durée,Prix Total
0,1,Music,3290,12981,3257.10
1,3,TV Shows,213,8239,423.87
2,5,90’s Music,1477,5914,1462.23
3,8,Music,3290,12981,3257.10
4,9,Music Videos,1,4,0.99
5,10,TV Shows,213,8239,423.87
6,11,Brazilian Music,39,141,38.61
7,12,Classical,75,329,74.25
8,13,Classical 101 - Deep Cuts,25,99,24.75
9,14,Classical 101 - Next Steps,25,117,24.75


Q05. Produire une liste des tous les genres de musique reprenant son genre, le nombre de pistes, la durée de toutes les pistes (en minutes), le coût total et le coût moyen (Liste triée par nom de genre)


In [34]:
query = """SELECT g.GenreId, g.Name as 'Genre', 
		COUNT(t.TrackId) AS 'Nombre de piste' , SUM(t.Milliseconds/60000) AS 'Durée', SUM(t.UnitPrice) AS 'Prix Total', 
		AVG(t.UnitPrice) as 'Prix moyen'		
FROM genres g
LEFT JOIN tracks t ON g.GenreId = t.GenreId
GROUP BY Genre"""

In [35]:
results = pd.read_sql_query(query, conn)
results

,GenreId,Genre,Nombre de piste,Durée,Prix Total,Prix moyen
0,23,Alternative,40,154,39.60,0.99
1,4,Alternative & Punk,332,1133,328.68,0.99
2,6,Blues,81,325,80.19,0.99
3,11,Bossa Nova,15,46,14.85,0.99
4,24,Classical,74,329,73.26,0.99
5,22,Comedy,17,442,33.83,1.99
6,21,Drama,64,2711,127.36,1.99
7,12,Easy Listening,24,64,23.76,0.99
8,15,Electronica/Dance,30,136,29.70,0.99
9,13,Heavy Metal,28,126,27.72,0.99


Q06. Indiquez les caractéristiques principales de toutes les pistes correspondant au genre 'Science Fiction' (nom de piste, album, artiste de l'album, compositeur, nom du type de media, durée (en secondes) et prix unitaire


In [38]:
query = """SELECT t.Name AS 'Piste', g.Name AS 'Genre', a2.Title AS 'ALBUM', t.Composer AS 'Compositeur',
		mt.Name AS 'Type', t.Milliseconds/1000 AS 'Durée en secondes', t.UnitPrice AS 'Prix'
FROM  tracks t
INNER JOIN genres g ON t.GenreId = g.GenreId
INNER JOIN media_types mt ON t.MediaTypeId = mt.MediaTypeId 
INNER JOIN albums a2 ON t.AlbumId = a2.AlbumId 
WHERE g.Name = 'Science Fiction'"""

In [39]:
results = pd.read_sql_query(query, conn)
results

,Piste,Genre,ALBUM,Compositeur,Type,Durée en secondes,Prix
0,Battlestar Galactica: The Story So Far,Science Fiction,Battlestar Galactica: The Story So Far,None,Protected MPEG-4 video file,2622,1.99
1,A Measure of Salvation,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2563,1.99
2,Hero,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2713,1.99
3,Unfinished Business,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2622,1.99
4,The Passage,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2623,1.99
5,The Eye of Jupiter,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2618,1.99
6,Rapture,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2624,1.99
7,Taking a Break from All Your Worries,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2624,1.99
8,The Woman King,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2626,1.99
9,A Day In the Life,Science Fiction,"Battlestar Galactica, Season 3",None,Protected MPEG-4 video file,2620,1.99


Q07. Retrouvez tous les clients (nom, prénom, email, pays) ayant acheté de la musique de type 'World'


In [41]:
query = """SELECT c.firstname || ' ' || c.lastname AS 'Customer', c.Email, c.Country, g2.Name AS 'Genre'
FROM customers c 
INNER JOIN invoices i2 ON c.CustomerId = i2.CustomerId 
INNER JOIN invoice_items ii ON i2.InvoiceId = ii.InvoiceId 
INNER JOIN tracks t2 ON ii.TrackId = t2.TrackId 
INNER JOIN genres g2 ON t2.GenreId = g2.GenreId 
WHERE g2.Name = 'World'"""

In [42]:
results = pd.read_sql_query(query, conn)
results

,Customer,Email,Country,Genre
0,João Fernandes,jfernandes@yahoo.pt,Portugal,World
1,Jennifer Peterson,jenniferp@rogers.ca,Canada,World
2,Mark Philips,mphilips12@shaw.ca,Canada,World
3,Steve Murray,steve.murray@yahoo.uk,United Kingdom,World
4,João Fernandes,jfernandes@yahoo.pt,Portugal,World
5,Jennifer Peterson,jenniferp@rogers.ca,Canada,World
6,Martha Silk,marthasilk@gmail.com,Canada,World
7,Roberto Almeida,roberto.almeida@riotur.gov.br,Brazil,World
8,Joakim Johansson,joakim.johansson@yahoo.se,Sweden,World
9,Aaron Mitchell,aaronmitchell@yahoo.ca,Canada,World


Q08. Nommez les 15 artistes comptant le plus de pistes de type 'Rock'


In [43]:
query = """SELECT g2.Name AS 'Genre',COUNT(t2.TrackId) AS 'Nombre de piste', a3.Name AS 'Artiste'
FROM genres g2 
LEFT JOIN tracks t2 ON g2.GenreId =t2.GenreId 
INNER JOIN albums a2 ON t2.AlbumId = a2.AlbumId 
INNER JOIN artists a3 ON a2.ArtistId = a3.ArtistId 
WHERE g2.Name = 'Rock'
GROUP BY a3.Name 
ORDER BY COUNT(t2.TrackId) DESC 
LIMIT 15"""

In [44]:
results = pd.read_sql_query(query, conn)
results

,Genre,Nombre de piste,Artiste
0,Rock,114,Led Zeppelin
1,Rock,112,U2
2,Rock,92,Deep Purple
3,Rock,81,Iron Maiden
4,Rock,54,Pearl Jam
5,Rock,52,Van Halen
6,Rock,45,Queen
7,Rock,41,The Rolling Stones
8,Rock,40,Creedence Clearwater Revival
9,Rock,35,Kiss


Q09. Nommez les 5 meilleurs clients 

In [46]:
query = """SELECT  DISTINCT(c.lastname), c.FirstName, SUM(i2.Total)
FROM invoices i2 
LEFT JOIN customers c ON i2.CustomerId = c.CustomerId 
GROUP BY c.LastName 
ORDER  BY SUM(i2.Total) DESC
LIMIT 5"""

In [47]:
results = pd.read_sql_query(query, conn)
results

,LastName,FirstName,SUM(i2.Total)
0,Holý,Helena,49.62
1,Cunningham,Richard,47.62
2,Rojas,Luis,46.62
3,O'Reilly,Hugh,45.62
4,Kovács,Ladislav,45.62


Q10. Détaillez (piste, album, artiste, genre et type media) tous les achats effectués par le client 'Richard Cunningham' (triée par date achat)


Q11. Produire la liste de tous les contacts de la base de données (clients & employés) avec leur nom, prénom, type (client ou employé), email, pays et no de téléphone


Q12. Produire la liste de répartition des ventes par pays (basée sur `BillingCountry`)